In [1]:
# Load dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, text
import pymc as pm
import arviz as az
from scipy.stats import zscore

# Define the SQLalchemy engine
engine = create_engine(f"sqlite:////Users/JO/PhD/neuro-ascertainment/data/db.sqlite")

# Read the SQL query from the file
with open('/Users/JO/PhD/neuro-ascertainment/candidate-queries/karolinska-nsicu-cohort/karolinska-cohort.sql', 'r') as file:
    query = file.read()

SEED = 20
rng = np.random.default_rng(SEED)

def count_id(df):
    LopNr = df['LopNr'].nunique() if 'LopNr' in df else 'Column missing'
    VtfId_LopNr = df['VtfId_LopNr'].nunique() if 'VtfId_LopNr' in df else 'Column missing'
    HADM_ID = df['HADM_ID'].nunique() if 'HADM_ID' in df else 'Column missing'
    return print(f'Unique patients: {LopNr} | Unique SIR admits: {VtfId_LopNr} | Unique PAR admits: {HADM_ID}')

In [2]:
query_PAR_HADM = query + "SELECT * FROM PAR_HADM"
PAR_HADM = pd.read_sql(query_PAR_HADM, engine)

In [3]:
PAR_HADM.head()

,LopNr,indexdatum1,indexdatum2,indexdatum3,indexdatum4,indexdatum5,indexdatum6,indexdatum7,indexdatum8,indexdatum9,...,OPD24,OPD25,OPD26,OPD27,OPD28,OPD29,OPD30,LK,slutrapporterad,HADM_ID
0,1.0,16134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,1
1,1.0,16134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,2
2,1.0,16134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,3
3,1.0,16134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,4
4,1.0,16134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,5


In [4]:
count_id(PAR_HADM)

Unique patients: 59333 | Unique SIR admits: Column missing | Unique PAR admits: 359305


In [5]:
query_K_ICU_ADMISSIONS = query + "SELECT * FROM K_ICU_ADMISSIONS"
K_ICU_ADMISSIONS = pd.read_sql(query_K_ICU_ADMISSIONS, engine)

In [6]:
K_ICU_ADMISSIONS.head()

,VtfId_LopNr,LopNr,InskrTidPunkt,UtskrTidPunkt,AvdNamn
0,175619.0,33177.0,1606887420,1607366520,S-CIVA
1,175620.0,33177.0,1607879820,1609839480,S-CIVA
2,175617.0,21183.0,1609025100,1609623000,S-CIVA
3,176206.0,33694.0,1610649900,1612279200,S-CIVA
4,176752.0,14417.0,1612186560,1614596400,S-CIVA


In [7]:
count_id(K_ICU_ADMISSIONS)

Unique patients: 6454 | Unique SIR admits: 7673 | Unique PAR admits: Column missing


In [8]:
query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR = query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR"
K_ICU_ADMISSIONS_MATCHED_WITH_PAR = pd.read_sql(query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR, engine)

In [9]:
K_ICU_ADMISSIONS_MATCHED_WITH_PAR.head(5)

,VtfId_LopNr,HADM_ID,LopNr,InskrTidPunkt,UtskrTidPunkt,AvdNamn,INDATUM,UTDATUM,MVO,SJUKHUS
0,175619.0,183658,33177.0,1606887420,1607366520,S-CIVA,18592,18604.0,301,11003
1,175619.0,183660,33177.0,1606887420,1607366520,S-CIVA,18607,18650.0,301,11003
2,175620.0,183660,33177.0,1607879820,1609839480,S-CIVA,18607,18650.0,301,11003
3,175617.0,117964,21183.0,1609025100,1609623000,S-CIVA,18622,18648.0,331,11003
4,176206.0,186729,33694.0,1610649900,1612279200,S-CIVA,18641,18668.0,331,11003


In [10]:
count_id(K_ICU_ADMISSIONS_MATCHED_WITH_PAR)

Unique patients: 5673 | Unique SIR admits: 6539 | Unique PAR admits: 7141


In [11]:
query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX = query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX"
K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX = pd.read_sql(query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX, engine)

In [12]:
K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX

,VtfId_LopNr,HADM_ID,LopNr,InskrTidPunkt,UtskrTidPunkt,AvdNamn,INDATUM,UTDATUM,MVO,SJUKHUS,DIAGNOS,OP,DX_GROUP
0,175619.0,183658,33177.0,1606887420,1607366520,S-CIVA,18592,18604.0,301,11003,S3280 S8290 S831 S2720 S2240 C900 I489 I109 N1...,DG017 DG023 DR029 GAA10 QAB00 TPH15 UGC12 ZV049,OTHER
1,175619.0,183660,33177.0,1606887420,1607366520,S-CIVA,18607,18650.0,301,11003,J969 J958A C900 I109 N189 I489 A410 Z930,DG012 DG017 DG023 TJD20 TJF10 TPH15 UDQ12 UGC1...,OTHER
2,175620.0,183660,33177.0,1607879820,1609839480,S-CIVA,18607,18650.0,301,11003,J969 J958A C900 I109 N189 I489 A410 Z930,DG012 DG017 DG023 TJD20 TJF10 TPH15 UDQ12 UGC1...,OTHER
3,175617.0,117964,21183.0,1609025100,1609623000,S-CIVA,18622,18648.0,331,11003,I608 S8260 I639 J9690 G819 R470 R139,AA006 AA011 AA045 AAF00 AAL00 AF068 AV130 AV13...,ASAH
4,176206.0,186729,33694.0,1610649900,1612279200,S-CIVA,18641,18668.0,331,11003,S065 S020 F102A Z930 R139 Z720A,AA011 AAA99 AAD00 AD009 AF021 AG009 AG040 AN03...,TBI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7926,145337.0,70328,12526.0,1475668260,1477704540,S-CIVA,17078,17102.0,131,11001,J809X J172 B599 M332 J969 E119 E039 I109 G473 ...,DT016 GBB00 XS918,OTHER
7927,144960.0,70328,12526.0,1475607660,1475638200,S-CIVA,17078,17102.0,131,11001,J809X J172 B599 M332 J969 E119 E039 I109 G473 ...,DT016 GBB00 XS918,OTHER
7928,146669.0,79829,14255.0,1482789960,1483287600,S-NIVA,17161,17172.0,121,11001,G001 J014 Z721 F319 F419,AAA99 AAF00 DG017 QD014 SI110 SP999 TAB00 TDM10,ABM
7929,146638.0,79829,14255.0,1482769200,1482789960,S-CIVA,17161,17172.0,121,11001,G001 J014 Z721 F319 F419,AAA99 AAF00 DG017 QD014 SI110 SP999 TAB00 TDM10,ABM


In [13]:
count_id(K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX)

Unique patients: 5673 | Unique SIR admits: 6539 | Unique PAR admits: 7141


In [14]:
## alla övrigt bort
## tag första PAR-DX
## Hierarki:
## TBI > SAH > AIS > ICH > ABM

## eller omvänt: först hieraki och sedan tid.
## skillnad i antal patienter??
## testa båda!!

In [15]:
query_TIME_HIERARCHY= query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX_TIME_HIERARCHY"
TIME_HIERARCHY  = pd.read_sql(query_TIME_HIERARCHY, engine)
TIME_HIERARCHY

,VtfId_LopNr,HADM_ID,LopNr,InskrTidPunkt,UtskrTidPunkt,AvdNamn,INDATUM,UTDATUM,MVO,SJUKHUS,DIAGNOS,OP,DX_GROUP,DX_ORDER
0,101236.0,184092,33266.0,1262790120,1262811780,S-CIVA,14615,14623,221,11001,G002 G060 G061 G062 G825 K746 E108,,ABM,1
1,101237.0,184092,33266.0,1262824440,1262876400,S-CIVA,14615,14623,221,11001,G002 G060 G061 G062 G825 K746 E108,,ABM,1
2,101239.0,108585,19456.0,1262948820,1262967300,S-CIVA,14617,14624,121,11001,G001 I639 J189,AA011 AG040 AAF00 AAW99,ABM,1
3,101240.0,216808,39070.0,1262972280,1264605840,S-CIVA,14617,14652,331,11001,S065 E107 N189 Z491 H540,AAD05 AAA99 AAA20 AAF00 GBB00 UJD02 JDB10,TBI,1
4,101242.0,110933,19891.0,1263226800,1263294000,S-CIVA,14620,14638,331,11001,I610,AAF00,ICH,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577,185981.0,139410,25209.0,1655454240,1657452300,S-CIVA,19158,19187,331,11003,G919 D339 I639 G822 J969 Z039,AA021 AAA99 AAB00 AAF00 AAK80 AF020 DG021 DV03...,HC,1
5578,185982.0,226781,40826.0,1656595800,1657719000,S-CIVA,19173,19193,331,11003,I607 G039 R509 I109 Z718,AA006 AA083 AAC10 AAF00 AAL00 AAL50 AF021 AG04...,ASAH,1
5579,185983.0,151729,27455.0,1655167800,1657823700,S-CIVA,19156,19194,331,11003,I614 I634 A047 J690 F412 R139 R252,AA083 AAA20 AAA99 AAF00 AAF00 AAL00 AAL15 AF02...,ASAH,1
5580,185984.0,74812,13327.0,1656960900,1658335200,S-CIVA,19177,19193,331,11003,S127 S140 J960 I460 J809X R572 N179 D649 D696 ...,AA045 ABC60 AF021 AF064 AG012 AN039 DG021 DR02...,CFX,1


In [16]:
small_d = TIME_HIERARCHY[['VtfId_LopNr','InskrTidPunkt', 'UtskrTidPunkt', 'HADM_ID', 'MVO', 'LopNr', 'INDATUM', 'UTDATUM', 'DX_GROUP', 'DIAGNOS', 'DX_ORDER']]
small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
small_d['InskrTidPunkt'] = pd.to_datetime(small_d['InskrTidPunkt'], unit='s').dt.normalize()
small_d['UtskrTidPunkt'] = pd.to_datetime(small_d['UtskrTidPunkt'], unit='s').dt.normalize()

filtered_df = small_d[small_d.groupby('VtfId_LopNr')['VtfId_LopNr'].transform('size') > 1]
filtered_df['timediff_to_sir'] = filtered_df['INDATUM'] - filtered_df['InskrTidPunkt']

/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2561574566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2561574566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2561574566.py:4: SettingWithCopyWarning: 
A value is trying

In [17]:
filtered_df

,VtfId_LopNr,InskrTidPunkt,UtskrTidPunkt,HADM_ID,MVO,LopNr,INDATUM,UTDATUM,DX_GROUP,DIAGNOS,DX_ORDER,timediff_to_sir
8,101249.0,2010-02-11,2010-02-15,239016,331,42938.0,2010-01-31,2010-02-15,CFX,S127,1,-11 days
9,101249.0,2010-02-11,2010-02-15,239017,221,42938.0,2010-02-15,2010-05-17,CFX,S140 S120,2,4 days
10,101250.0,2010-01-22,2010-01-31,239015,303,42938.0,2010-01-21,2010-01-31,CFX,S120 S121 J189,1,-1 days
11,101250.0,2010-01-22,2010-01-31,239016,331,42938.0,2010-01-31,2010-02-15,CFX,S127,2,9 days
19,101267.0,2010-02-13,2010-02-14,218208,121,39334.0,2010-02-13,2010-02-14,ABM,G039 I109 S021,1,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...
5560,185867.0,2022-06-17,2022-06-25,152322,331,27572.0,2022-06-19,2022-07-06,TBI,S066 S062 S368 J960 J690 S069 Z598 S224 T009 T...,2,2 days
5563,185941.0,2022-06-28,2022-06-30,152321,301,27572.0,2022-06-17,2022-06-19,TBI,S0660 S0620 S0650 S2730 S2240 S2700 S017,1,-11 days
5564,185941.0,2022-06-28,2022-06-30,152322,331,27572.0,2022-06-19,2022-07-06,TBI,S066 S062 S368 J960 J690 S069 Z598 S224 T009 T...,2,-9 days
5574,185972.0,2022-07-05,2022-07-07,40154,221,7036.0,2022-07-05,2022-07-05,ASAH,I603 I671 I639 I489 I509 J449,1,0 days


In [18]:
check_vtf = rng.choice(filtered_df['VtfId_LopNr'], 1)
filtered_df.query(f"VtfId_LopNr == {check_vtf}")

,VtfId_LopNr,InskrTidPunkt,UtskrTidPunkt,HADM_ID,MVO,LopNr,INDATUM,UTDATUM,DX_GROUP,DIAGNOS,DX_ORDER,timediff_to_sir
5201,181018.0,2021-09-02,2021-10-05,117779,301,21159.0,2021-09-02,2021-09-03,TBI,S066 S066 S0630 S1210 S0210 S001 S0650 I620,1,0 days
5202,181018.0,2021-09-02,2021-10-05,117780,331,21159.0,2021-09-03,2021-10-05,TBI,S066 S066 S1210 S1210 S001 S001 S0650 S0650 S0...,2,1 days


In [19]:
count_id(TIME_HIERARCHY)

Unique patients: 4150 | Unique SIR admits: 4806 | Unique PAR admits: 4965


In [20]:
query_HIERARCHY_TIME= query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX_HIERARCHY_TIME"
HIERARCHY_TIME  = pd.read_sql(query_HIERARCHY_TIME, engine)
HIERARCHY_TIME

,VtfId_LopNr,HADM_ID,LopNr,InskrTidPunkt,UtskrTidPunkt,AvdNamn,INDATUM,UTDATUM,MVO,SJUKHUS,DIAGNOS,OP,DX_GROUP,DX_ORDER
0,101236.0,184092,33266.0,1262790120,1262811780,S-CIVA,14615,14623,221,11001,G002 G060 G061 G062 G825 K746 E108,,ABM,1
1,101237.0,184092,33266.0,1262824440,1262876400,S-CIVA,14615,14623,221,11001,G002 G060 G061 G062 G825 K746 E108,,ABM,1
2,101239.0,108585,19456.0,1262948820,1262967300,S-CIVA,14617,14624,121,11001,G001 I639 J189,AA011 AG040 AAF00 AAW99,ABM,1
3,101240.0,216808,39070.0,1262972280,1264605840,S-CIVA,14617,14652,331,11001,S065 E107 N189 Z491 H540,AAD05 AAA99 AAA20 AAF00 GBB00 UJD02 JDB10,TBI,1
4,101242.0,110933,19891.0,1263226800,1263294000,S-CIVA,14620,14638,331,11001,I610,AAF00,ICH,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577,185981.0,139410,25209.0,1655454240,1657452300,S-CIVA,19158,19187,331,11003,G919 D339 I639 G822 J969 Z039,AA021 AAA99 AAB00 AAF00 AAK80 AF020 DG021 DV03...,HC,1
5578,185982.0,226781,40826.0,1656595800,1657719000,S-CIVA,19173,19193,331,11003,I607 G039 R509 I109 Z718,AA006 AA083 AAC10 AAF00 AAL00 AAL50 AF021 AG04...,ASAH,1
5579,185983.0,151729,27455.0,1655167800,1657823700,S-CIVA,19156,19194,331,11003,I614 I634 A047 J690 F412 R139 R252,AA083 AAA20 AAA99 AAF00 AAF00 AAL00 AAL15 AF02...,ASAH,1
5580,185984.0,74812,13327.0,1656960900,1658335200,S-CIVA,19177,19193,331,11003,S127 S140 J960 I460 J809X R572 N179 D649 D696 ...,AA045 ABC60 AF021 AF064 AG012 AN039 DG021 DR02...,CFX,1


In [21]:
small_d.query("DX_ORDER == 1").DX_GROUP.value_counts()

DX_GROUP
ASAH    1344
TBI     1249
ICH      657
AIS      468
CFX      297
ABM      217
SEP      162
TUM      135
SDH      124
HC        53
AVM       42
ENC       40
CVT       18
Name: count, dtype: int64

In [22]:
count_id(HIERARCHY_TIME)

Unique patients: 4150 | Unique SIR admits: 4806 | Unique PAR admits: 4965


In [23]:
small_d = HIERARCHY_TIME[['VtfId_LopNr','InskrTidPunkt', 'UtskrTidPunkt', 'HADM_ID', 'MVO', 'LopNr', 'INDATUM', 'UTDATUM', 'DX_GROUP', 'DIAGNOS', 'DX_ORDER']]
small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
small_d['InskrTidPunkt'] = pd.to_datetime(small_d['InskrTidPunkt'], unit='s').dt.normalize()
small_d['UtskrTidPunkt'] = pd.to_datetime(small_d['UtskrTidPunkt'], unit='s').dt.normalize()

filtered_df = small_d[small_d.groupby('VtfId_LopNr')['VtfId_LopNr'].transform('size') > 1]
filtered_df['timediff_to_sir'] = filtered_df['INDATUM'] - filtered_df['InskrTidPunkt']

/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2839594063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2839594063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2839594063.py:4: SettingWithCopyWarning: 
A value is trying

In [24]:
check_vtf = rng.choice(filtered_df['VtfId_LopNr'], 1)
filtered_df.query(f"VtfId_LopNr == {check_vtf}")

,VtfId_LopNr,InskrTidPunkt,UtskrTidPunkt,HADM_ID,MVO,LopNr,INDATUM,UTDATUM,DX_GROUP,DIAGNOS,DX_ORDER,timediff_to_sir
1796,132411.0,2014-02-10,2014-02-14,68368,331,12135.0,2014-02-10,2014-02-16,ABM,G060 I489 I109 Z958 I340 J324,1,0 days
1797,132411.0,2014-02-10,2014-02-14,68370,121,12135.0,2014-02-20,2014-03-24,ABM,G060,2,10 days


In [25]:
small_d.query("DX_ORDER == 1").DX_GROUP.value_counts()

DX_GROUP
ASAH    1346
TBI     1251
ICH      661
AIS      471
CFX      296
ABM      217
SEP      163
TUM      135
SDH      119
HC        48
AVM       42
ENC       41
CVT       16
Name: count, dtype: int64

In [26]:
only_first = small_d.groupby("LopNr").apply(lambda x: x.sort_values(by="InskrTidPunkt").head(1))

/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2197635333.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  only_first = small_d.groupby("LopNr").apply(lambda x: x.sort_values(by="InskrTidPunkt").head(1))


In [27]:
count_id(only_first)

Unique patients: 4150 | Unique SIR admits: 4150 | Unique PAR admits: 4150


In [28]:
only_first.DX_GROUP.value_counts()

DX_GROUP
ASAH    1191
TBI     1062
ICH      578
AIS      425
CFX      253
ABM      173
TUM      122
SEP      121
SDH      105
HC        38
ENC       35
AVM       32
CVT       15
Name: count, dtype: int64

In [29]:
only_first_asah = only_first.query("DX_GROUP == 'ASAH'") 
only_first_asah['IVA_tid'] = only_first_asah['UtskrTidPunkt'] - only_first_asah['InskrTidPunkt']
only_first_asah['IVA_tid'].mean()

/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/2128328699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_first_asah['IVA_tid'] = only_first_asah['UtskrTidPunkt'] - only_first_asah['InskrTidPunkt']


Timedelta('8 days 15:50:19.647355163')

In [30]:
i671 = only_first[only_first.DIAGNOS.str[:4] == 'I671']
i671['IVA_tid'] = i671['UtskrTidPunkt'] - i671['InskrTidPunkt']
i671['IVA_tid'].mean()

/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_19034/1240397002.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i671['IVA_tid'] = i671['UtskrTidPunkt'] - i671['InskrTidPunkt']


Timedelta('3 days 13:34:44.210526315')

In [31]:
i671['InskrTidPunkt'].dt.year.value_counts()

InskrTidPunkt
2015    15
2011    11
2014     9
2013     7
2010     7
2018     7
2012     5
2016     5
2017     4
2020     3
2022     1
2021     1
2019     1
Name: count, dtype: int64

In [32]:
(1191-76)/12.5

89.2

In [33]:
query_DESC_SIR = query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX_TIME_HIERARCHY K LEFT JOIN DESCRIPTIVE_SIR D ON K.VtfId_LopNr = D.VtfId_LopNr"
DESC_SIR  = pd.read_sql(query_DESC_SIR, engine)
DESC_SIR

,VtfId_LopNr,HADM_ID,LopNr,InskrTidPunkt,UtskrTidPunkt,AvdNamn,INDATUM,UTDATUM,MVO,SJUKHUS,...,SAPS_min_SBP,SAPS_max_HR,SAPS_hypotension,hemodynamic_instability_markers,SOFA_high_norepi_dose,SAPS_total_score,SAPS_min_pH,SAPS_max_temp,SAPS_acidosis,SAPS_hypothermia
0,101236.0,184092,33266.0,1262790120,1262811780,S-CIVA,14615,14623,221,11001,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,101237.0,184092,33266.0,1262824440,1262876400,S-CIVA,14615,14623,221,11001,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,101239.0,108585,19456.0,1262948820,1262967300,S-CIVA,14617,14624,121,11001,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
3,101240.0,216808,39070.0,1262972280,1264605840,S-CIVA,14617,14652,331,11001,...,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN
4,101242.0,110933,19891.0,1263226800,1263294000,S-CIVA,14620,14638,331,11001,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577,185981.0,139410,25209.0,1655454240,1657452300,S-CIVA,19158,19187,331,11003,...,126.0,103.0,0.0,0,0,75.0,7.52,37.9,0.0,0.0
5578,185982.0,226781,40826.0,1656595800,1657719000,S-CIVA,19173,19193,331,11003,...,177.0,80.0,0.0,0,0,36.0,7.36,35.9,0.0,0.0
5579,185983.0,151729,27455.0,1655167800,1657823700,S-CIVA,19156,19194,331,11003,...,165.0,64.0,0.0,1,0,33.0,7.48,39.0,0.0,0.0
5580,185984.0,74812,13327.0,1656960900,1658335200,S-CIVA,19177,19193,331,11003,...,113.0,80.0,0.0,1,0,58.0,7.35,36.1,0.0,0.0


In [34]:
count_id(DESC_SIR)

Unique patients: 4150 | Unique SIR admits: VtfId_LopNr    4806
VtfId_LopNr    4806
dtype: int64 | Unique PAR admits: 4965


In [35]:
query_DESC_PAR = query + "SELECT * FROM SUMMARY_TABLE"
DESC = pd.read_sql(query_DESC_PAR, engine)
DESC

DESC['par_adm_date'] = pd.to_datetime(DESC['par_adm_date'], unit='D')
DESC['par_dsc_date'] = pd.to_datetime(DESC['par_dsc_date'], unit='D')
DESC['sir_adm_time'] = pd.to_datetime(DESC['sir_adm_time'], unit='s').dt.normalize()
DESC['sir_dsc_time'] = pd.to_datetime(DESC['sir_dsc_time'], unit='s').dt.normalize()

OperationalError: (sqlite3.OperationalError) near "SUMMARY_TABLE": syntax error
[SQL: -- Redefine the PAR table adding a unique HADM_ID
-- Keep only admissions where the patient was >= 18 yrs at admission
-- Keep only admissions after 2010/01/01
WITH PAR_HADM AS (
    SELECT *,
           ROW_NUMBER() OVER ( 
               ORDER BY LopNr,
                        INDATUM,
                        UTDATUM,
                        CASE WHEN SJUKHUS NOT IN ('11001', '11003', '51001', '12001', '21001', '64001', '41001', '41002') THEN 0 ELSE 1 END
           ) AS HADM_ID
    FROM PAR
    WHERE 
        ALDER >= 18
        AND INDATUM >= 14610 -- This is 2010/01/01
)
,

-- Define all ICU admission to a K ICU (CIVA/NIVA)
K_ICU_ADMISSIONS AS (
    SELECT
        S.VtfId_LopNr,
        S.LopNr,
        S.InskrTidPunkt,
        S.UtskrTidPunkt,
        S.AvdNamn
    FROM SIR_BASDATA S
    WHERE S.AvdNamn IN ('S-CIVA','S-NIVA')
),

-- Define a window with all ICU admissions in K_ICU_ADMISSIONS
-- 

In [ ]:
DESC.columns

Index(['LopNr', 'VtfId_LopNr', 'HADM_ID', 'par_tertiary_center',
       'par_adm_date', 'par_dsc_date', 'sex_female', 'age', 'DX_GROUP',
       'DX_ORDER', 'VtfId_LopNr', 'sir_icu_name', 'sir_hospital_type',
       'sir_adm_time', 'sir_dsc_time', 'sir_total_time', 'admission_height',
       'admission_weight', 'BMI', 'DNR', 'SAPS_GCS', 'SAPS_GCSm', 'SAPS_RLS85',
       'SOFA_worst_RLS85', 'SOFA_worst_GCS', 'SOFA_worst_GCSm',
       'overall_worst_RLS85', 'overall_worst_GCS', 'overall_worst_GCSm',
       'SAPS_obtunded', 'overall_obtunded', 'SAPS_unconcious',
       'overall_unconcious', 'sir_consciousness_level', 'SAPS_AMV', 'KVA_IMV',
       'KVA_NIV', 'any_AMV', 'SAPS_PFI', 'SAPS_PAO2', 'SAPS_hypoxia', 'ARDS',
       'respiratory_instability_markers', 'SAPS_min_SBP', 'SAPS_max_HR',
       'SAPS_hypotension', 'hemodynamic_instability_markers',
       'SOFA_high_norepi_dose', 'SAPS_total_score', 'SAPS_min_pH',
       'SAPS_max_temp', 'SAPS_acidosis', 'SAPS_hypothermia'],
      dtype='o

In [ ]:
count_id(DESC)

Unique patients: 4150 | Unique SIR admits: VtfId_LopNr    4806
VtfId_LopNr    4806
dtype: int64 | Unique PAR admits: 4290


In [ ]:
DESC.groupby('LopNr').sir_adm_time.count().sort_values(ascending=False)

LopNr
40632.0    7
7733.0     5
46103.0    5
12603.0    5
45590.0    4
          ..
21254.0    1
21264.0    1
21275.0    1
21278.0    1
64562.0    1
Name: sir_adm_time, Length: 4150, dtype: int64

In [ ]:
DESC[DESC['LopNr'] == 21068.0][['LopNr', 'VtfId_LopNr', 'HADM_ID', 'par_adm_date', 'par_dsc_date', 'sir_adm_time', 'sir_dsc_time', 'DX_GROUP']]

,LopNr,VtfId_LopNr,VtfId_LopNr,HADM_ID,par_adm_date,par_dsc_date,sir_adm_time,sir_dsc_time,DX_GROUP
1427,21068.0,158229.0,158229.0,117289,2018-07-22,2018-08-07,2018-07-22,2018-08-01,ICH


In [ ]:
DESC[DESC['LopNr'] == 40632.0][['LopNr', 'VtfId_LopNr', 'HADM_ID', 'par_adm_date', 'par_dsc_date', 'sir_adm_time', 'sir_dsc_time', 'DX_GROUP']]

,LopNr,VtfId_LopNr,VtfId_LopNr,HADM_ID,par_adm_date,par_dsc_date,sir_adm_time,sir_dsc_time,DX_GROUP
1532,40632.0,130460.0,130460.0,225561,2014-08-18,2014-08-19,2014-08-18,2014-08-19,SEP
1550,40632.0,131184.0,131184.0,225563,2014-09-26,2014-09-29,2014-09-26,2014-09-27,SEP
1799,40632.0,133748.0,133748.0,225564,2015-01-15,2015-01-17,2015-01-16,2015-01-16,SEP
1985,40632.0,138061.0,138061.0,225567,2015-04-09,2015-04-11,2015-04-10,2015-04-10,SEP
2233,40632.0,140247.0,140247.0,225569,2015-12-12,2015-12-15,2015-12-12,2015-12-13,SEP
3972,40632.0,170506.0,170506.0,225572,2020-02-07,2020-02-10,2020-02-07,2020-02-07,SEP
4123,40632.0,173306.0,173306.0,225573,2020-08-29,2020-08-31,2020-08-29,2020-08-29,SEP
